# From Data to Predictions

This notebook goes through the steps to build and train a Solar Wind Plasma Sheet Neural Network (SWPSNN) from scratch, presuming that we have zero data to start with. In this example, we will not be creating the full model, but only an example model using a small amount of data. However, the steps included here are all that is required to reproduce the full model.

For this example, we will be training and testing on data from the most recent THEMIS tail period April 1, 2021 - October 31, 2021.

(The full model in the paper uses data inclusive of years 2008-2020.)







## Outline
* [0. Import Tools](#imports)
* [1. Download Data](#download)
 * [1.1 Download THEMIS Data](#themis_download)
 * [1.2 Download OMNI Data](#omni_download)
 * [1.3 Download FISM-2 Data](#fism_download)
* [2. Preprocess Data](#preprocess)
 * [2.1 Filter THEMIS Data](#filter_themis)
 * [2.2 Create Target Arrays](#targets)
 * [2.3 Build Feature Arrays](#features)
   * [2.3.1 Handle OMNI Data](#handle_omni)
   * [2.3.2 Calculate SW-Mag Coupling Functions](#calc_swmag_cfns)
   * [2.3.3 IMF $B_Z$ ULF Wave Power](#imfbzpow)
   * [2.3.4 Concatenate Features](#concat_features)
 * [2.4 Model-Ready Data](#model_ready_data)
* [3. Artificial Neural Network](#neural_network)
 * [3.1 Hyperparameter Optimization](#optimization)
 * [3.2 Model Training](#nn_training)
* [4. Make Predictions](#predictions)
* [5. Model Assessment](#assessment)
 * [5.1 Metrics](#metrics)
 * [5.2 Figures](#figures)

<a id='imports'></a>
## 0. Import Tools
Much of the heavy lifting is accomplished by functions and methods external to this notebook. 

In [1]:
# Importing required packages.
import pandas as pd
import traindata as td
import util
from nnconstants import *

<a id='download'></a>
## 1. Download Data
We downloaded and read the THEMIS and OMNI NASA CDF files from [NASA CDAWeb direct https to data](https://cdaweb.gsfc.nasa.gov/pub/data/).

FISM-2 data are downloaded from the [LISIRD LaTiS API](https://lasp.colorado.edu/lisird/about/latis).

To read the CDF files we use the [spacepy](https://github.com/spacepy/spacepy) module, which requires several dependencies in order to read the files, including the [NASA CDF library](https://cdf.gsfc.nasa.gov/html/sw_and_docs.html).

Downloading all of the data files will take several minutes.

In [2]:
# Data between which dates (inclusive), in yyyymmdd format.
day1, day2 = '20210401', '20211031'

<a id='themis_download'></a>
### 1.1 Download THEMIS Data

We extract the data from the CDF files. The extract data method will download the CDF file from source if necessary. Our extracted data are stored in `pandas DataFrames`. The `DataFrame` combines data from the gmom and ssc THEMIS CDFs.

Some of the CDFs for specific days and probes may not exist. If so, the `extract_data` method prints a message to screen.

In [3]:
# Specify which THEMIS spacecraft we want.
probes = ['tha', 'thd', 'the']

# The downloading and extracting methods belong to a 
# ThemisData object.
tdata = td.ThemisData()

# The extract_data() method will first call the download_cdf() method,
# if it cannot find the THEMIS CDF file.
tdata.extract_data(
    dates=(day1, day2),
    probes=probes,
    )

<a id='omni_download'></a>
### 1.2 Download OMNI Data

We extract the OMNI data from the OMNI CDF files. If the CDF files are not in the expected folder, they are downloaded from CDAWeb. We used High Resolution OMNI version 1 at 1-minute resolution.

In [4]:
omni_data = td.OmniData()

omni_first_month = '20210301'
omni_last_month = day2
# This method reads the CDF files (first downloading if necessary),
# extracts the desired data, and stores them into a pandas DataFrame.
omni_data.extract_data(
    dates=(omni_first_month, omni_last_month),
    # All other parameters are set by default.
    )

<a id='fism_download'></a>
### 1.3 Download FISM-2 Data

In [5]:
# The FISM API requires using full time stamp information.
beg_time = pd.Timestamp(2021, 3, 1, 0, 0)
end_time = pd.Timestamp(2021, 10, 31, 23, 59)

# Which band to download.
band54 = 'E54_0_65_0'

fism_data = td.FismData()

fism_data.download_fism_data(
    beg_time,
    end_time, 
    band54,
    )


In [6]:
# The downloaded FISM data is in a single file. 
# We read it and save it to a pandas DataFrame.
fism_data.extract_fism_data(
    beg_time,
    end_time,
    )

<a id='Preprocess Data'></a>
## 2. Preprocess Data



In [7]:
# We combine the daily dataframes of the THEMIS data into a single dataset.
# This method will print an error for each file that it cannot find.
# The combine_dfs method also filters out bad data based on data quality flags.
tdata = td.ThemisData()
tdata.combine_dfs(
    day1,
    day2,
    save_dir=TRAINING_DATA_FLDR,
    data_dir=THEMIS_DATA_FLDR,
    dataq=GOOD_FLAGS)

<a id='filter_themis'></a>
### 2.1 Filter THEMIS Data

We filter the data by restricting the data to our spatial domain and by using a plasma $\beta > 1$ criterion for the plasma sheet.

This also takes the $\log_{10}$ of the flux values.

In [8]:
tdata = td.ThemisData()

combo_themis_fname = './Data/Training/themis_' + day1 + '_' + day2 + '.pkl'

combined_themis = pd.read_pickle(combo_themis_fname)

plasmasheet_themis = tdata.filter_data(
    combined_themis,
    beta_threshold=1,
    keep_greater_beta=True,
    mlt_bounds=(18, 6),
    rdist_bounds=(6, 12),
    drop_extra_cols=False,
    )

# Save the result to disk -- these are now THEMIS data filtered to the model domain.
plasmasheet_themis.to_pickle('./Data/Training/themis_plasmasheet.pkl')

In [9]:
plasmasheet_themis.head()

,82.5,108.7,142.5,187.4,247.0,324.7,427.6,562.1,740.4,975.4,...,139000.0,203500.0,beta,mlt_value,X_GSM,Y_GSM,Z_GSM,probe,mlt,rdist
2021-04-01 00:50:00,6.883753,6.842377,6.742959,6.592362,6.410459,6.233401,6.059652,7.657317,7.029628,7.246519,...,3.198986,3.079893,1.114652,5.859253,-0.432956,-11.744651,-5.244736,tha,5.859253,11.752629
2021-04-01 01:04:00,6.203963,6.410030,6.585228,6.734181,6.859944,6.955817,7.030360,7.086112,7.142532,7.206189,...,4.362095,3.867850,1.209154,5.896970,-0.319027,-11.824627,-5.225116,tha,5.896969,11.828930
2021-04-01 01:05:00,6.169248,6.392722,6.563641,6.707239,6.826357,6.917738,7.002177,7.060954,7.125762,7.202750,...,4.539483,4.131174,2.203031,5.899640,-0.310901,-11.830212,-5.223361,tha,5.899640,11.834296
2021-04-01 01:07:00,6.946283,7.055636,7.123389,7.156295,7.159882,7.142710,7.125055,7.105008,7.103677,7.127106,...,4.550329,4.092888,1.198978,5.904977,-0.294635,-11.841334,-5.219697,tha,5.904978,11.844999
2021-04-03 04:03:00,5.971495,6.140481,6.280296,6.394923,6.478318,6.563065,6.667162,6.786106,6.934462,7.089437,...,2.795789,0.562001,1.039242,5.314820,-1.852062,-10.213862,-2.590719,the,5.314822,10.380420


<a id='targets'></a>
### 2.2. Create Target Arrays
We create the target arrays that are used for training and testing (after appropriate train/val/test split).

In [10]:
# Start with the plasmasheet data.
target_df = plasmasheet_themis.copy()

# Remove energy channels higher than 100keV.
target_df.drop(
        ['139000.0', '203500.0'],
        axis=1,
        inplace=True,
        )

# Drop rows with any NaNs (possibly NaNs entered from previous steps).
target_df.dropna(axis=0, how='any', inplace=True)

# Save the position data (will need to build feature arrays).
position_fname = './Data/Training/themis_ps_locations.pkl'
position_data = target_df.loc[:, ['mlt', 'rdist', 'probe', 'beta']]
position_data.to_pickle(position_fname)

# Remove all non-flux (metadata columns).
metadata_columns = ['beta', 'mlt_value', 'X_GSM', 'Y_GSM', 'Z_GSM', 'probe', 'mlt', 'rdist']
target_df.drop(metadata_columns, axis=1, inplace=True)

# We converted to number flux from energy flux (not necessary for model, this was simply our preference).
target_df = util.convert_eflux_to_nflux(
    target_df,
    flux_log10=True,
    )

target_df_fname = './Data/Training/target_df.pkl'
target_df.to_pickle(target_df_fname)

In [11]:
target_df.head()

,82.5,108.7,142.5,187.4,247.0,324.7,427.6,562.1,740.4,975.4,...,26388.6,27000.0,28000.0,29000.0,30000.0,31000.0,41000.0,52000.0,65500.0,93000.0
2021-04-01 00:50:00,7.967299,7.806148,7.589144,7.319592,7.017762,6.721918,6.428614,7.907503,7.160161,7.257336,...,4.924671,4.850254,4.487048,4.176821,3.912929,3.712757,2.506895,1.079356,1.510462,1.200330
2021-04-01 01:04:00,7.287509,7.373801,7.431413,7.461412,7.467247,7.444335,7.399323,7.336298,7.273065,7.217007,...,4.260217,4.236028,4.145539,4.071344,4.012485,3.968362,3.265259,2.963595,2.688367,2.494585
2021-04-01 01:05:00,7.252794,7.356493,7.409826,7.434469,7.433660,7.406256,7.371139,7.311140,7.256295,7.213568,...,4.487550,4.459752,4.344590,4.240616,4.147213,4.063868,3.455919,3.170270,2.907100,2.678175
2021-04-01 01:07:00,8.029829,8.019406,7.969575,7.883526,7.767185,7.631228,7.494017,7.355195,7.234211,7.137924,...,3.850713,3.842810,3.850249,3.878762,3.936180,4.088925,3.507519,3.209467,2.993605,2.711093
2021-04-03 04:03:00,7.055041,7.104251,7.126481,7.122154,7.085621,7.051583,7.036125,7.036293,7.064996,7.100255,...,4.933073,4.843936,4.716270,4.608952,4.519248,4.444340,3.656834,3.218516,2.909599,2.067011


<a id='features'></a>
### 2.3. Build Feature Arrays

After the target arrays are built, we can build the feature arrays. We will use the same timestamps that are in the `target_df` and the associated `position_data`. 

First, we need to pre-process the OMNI data.

<a id='handle_omni'></a>
#### 2.3.1 Handle OMNI Data

We concatenate the monthly OMNI DataFrames into a single DataFrame and in the process convert the missing data fill values to NaN.

In [12]:
omni_data = td.OmniData()

omni_dir = './Data/Omni/hro_1min/'
start_month, end_month = '20210301', '20211001'

omni_concatenated = omni_data.concatenate_omni(
    omni_dir,
    omni_dir,
    start_month,
    end_month,
    )



In [13]:
omni_concatenated.head()

,BX_GSM,BY_GSM,BZ_GSM,Tp,Beta,flow_pressure,Efield,flow_speed,Np,VX_GSE,VY_GSE,VZ_GSE,BSNx_GSE,BSNy_GSE,BSNz_GSE
2021-03-01 00:00:00,-1.67,9.12,-1.92,38440.0,1.67,6.55,0.75,388.100006,21.740000,-388.000000,-0.4,-6.3,10.47,-0.71,0.30
2021-03-01 00:01:00,-2.10,9.11,-1.89,40459.0,1.45,5.84,0.74,391.500000,19.059999,-391.200012,-5.3,-12.7,10.48,-0.69,0.35
2021-03-01 00:02:00,-1.65,9.36,-1.16,34323.0,1.37,5.70,0.45,390.600006,18.690001,-390.200012,-4.7,-15.8,10.57,-0.69,0.42
2021-03-01 00:03:00,-1.02,9.47,-0.05,33298.0,1.35,5.63,0.02,390.600006,18.450001,-390.200012,-4.0,-15.4,10.76,-0.77,0.35
2021-03-01 00:04:00,-1.66,9.26,-0.49,31206.0,1.25,5.30,0.19,391.100006,17.320000,-390.899994,-0.9,-11.6,10.67,-0.81,0.27


<a id='calc_swmag_cfns'></a>
#### 2.3.2 Calculate SW-Mag Coupling Functions

We calculate Solar Wind -- Magnetosphere coupling functions using the concatenated OMNI dataframe. Any NaN values are propagated to the coupling functions.

In [14]:
omni_with_swmagcouple = omni_concatenated.copy()

# Add the palpha1 function.
omni_with_swmagcouple.loc[:, 'Palpha1'] = \
    omni_data.calc_Palpha1(
        omni_concatenated.BX_GSM,
        omni_concatenated.BY_GSM,
        omni_concatenated.BZ_GSM,
        omni_concatenated.flow_speed,
        omni_concatenated.Np,
        )

# Add VBs.
omni_with_swmagcouple.loc[:, 'E_geoeff'] = \
    omni_data.calc_geoeffective_efield(
        omni_concatenated.BZ_GSM,
        omni_concatenated.VX_GSE,
        )

# Add Narmax derived function.
omni_with_swmagcouple.loc[:, 'Narmax_CF'] = \
    omni_data.calc_narmax_cf(
        omni_concatenated.flow_pressure,
        omni_concatenated.flow_speed,
        omni_concatenated.BY_GSM,
        omni_concatenated.BZ_GSM,
        )

# Save to disk.
omni_with_swmagcouple.to_pickle("./Data/Omni/omni_swmag_coupling.pkl")

/Users/swigerbr/anaconda3/envs/swpsnn/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/swigerbr/anaconda3/envs/swpsnn/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sin
  result = getattr(ufunc, method)(*inputs, **kwargs)


<a id='imfbzpow'></a>
#### 2.3.3 IMF $B_Z$ ULF Wave Power

Calculating the ULF wave power may take several minutes depending on the length of the time series.

In [15]:
imfbz_ulf_power, imfbz_ulf_spectra = \
    omni_data.calc_ulf_wave_power(
        omni_concatenated.BZ_GSM,
        show_remaining_time=False, # Change to True if you lack patience.
        )



<a id='concat_features'></a>
#### 2.3.4. Concatenate Features

This is the step where all of the driving features are combined into a single DataFrame. We  interpolate over missing data, and normalize the features.

To normalize the features, we will use the same sample means and standard deviations that we used for the full model. These were calculated with the data from 2008-2020.

We drop extra data columns in the OMNI dataframe that were not used as features to the model.

In [16]:
# Loading previously saved data.
feature_data_stats = pd.read_pickle('./ExampleData/feature_data_stats.pkl')
fism_data = pd.read_pickle(
    './Data/Fism/fism2_2021-03-01T00:00:00.000Z_2021-10-31T23:59:00.000Z_E54_0_65_0.pkl'
    )



In [17]:
# Dropping extraneous columns.
extra_cols = ['BX_GSM', 'VX_GSE', 'VY_GSE', 'VZ_GSE', 'Tp', 'Beta', 'flow_pressure',
               'Efield', 'BSNx_GSE', 'BSNy_GSE', 'BSNz_GSE']
omni_with_swmagcouple.drop(
    extra_cols,
    axis=1,
    inplace=True,
    )

In [18]:
# Interpolating the remaining data.
omni_interpolated = omni_data.interpolate_omni(
    data=omni_with_swmagcouple,
    method='linear',
    )

In [19]:
# Adding imfbz wave power and fism to the dataframe.
# The fism data are at 5 minute. We resample to 1 minute.
omni_interpolated['fism_20eV'] = fism_data.twenty_eV.resample('1min').ffill()
omni_interpolated['bzpow'] = imfbz_ulf_power
    

In [20]:
# Normalizing the data.
from numpy import zeros
features_normed = pd.DataFrame(
    zeros(omni_interpolated.shape),
    index=omni_interpolated.index,
    columns=omni_interpolated.columns
    )

for feature_name in omni_interpolated.columns:
    sample_mean = feature_data_stats.loc[feature_name, 'mean']
    sample_std = feature_data_stats.loc[feature_name, 'std']
    
    features_normed.loc[:, feature_name] = \
        (omni_interpolated.loc[:, feature_name] - sample_mean) / sample_std
    
# We add back the mean/std to Egeoeff so that this feature remains rectified 
# (i.e., no positive values).
features_normed.E_geoeff += \
    feature_data_stats.at['E_geoeff', 'mean'] / feature_data_stats.at['E_geoeff', 'std']

feature_normed_fname = './Data/Training/features_normed.pkl'
features_normed.to_pickle(feature_normed_fname)

In [21]:
features_normed.head()

,BY_GSM,BZ_GSM,flow_speed,Np,Palpha1,E_geoeff,Narmax_CF,fism_20eV,bzpow
2021-03-01 00:00:00,2.606709,-0.637676,-0.270958,3.344645,2.729830,-0.924056,3.381525,-0.187259,NaN
2021-03-01 00:01:00,2.603876,-0.627640,-0.234643,2.772719,2.697888,-0.917120,3.244598,-0.187259,NaN
2021-03-01 00:02:00,2.674698,-0.383423,-0.244256,2.693759,3.388577,-0.561450,4.459871,-0.187259,NaN
2021-03-01 00:03:00,2.705859,-0.012079,-0.244256,2.642542,4.518154,-0.024200,6.688500,-0.187259,NaN
2021-03-01 00:04:00,2.646369,-0.159278,-0.238915,2.401394,3.922619,-0.237589,5.418993,-0.187259,NaN


In [22]:
features_normed.describe()

,BY_GSM,BZ_GSM,flow_speed,Np,Palpha1,E_geoeff,Narmax_CF,fism_20eV,bzpow
count,338812.000000,338812.000000,338729.000000,338729.000000,338714.000000,338714.000000,338714.000000,352796.000000,352770.000000
mean,0.025849,-0.035684,-0.209687,0.180823,-0.057861,-0.584259,-0.044527,0.092923,-0.055514
std,1.000494,1.015874,0.783392,1.145094,0.886387,0.932804,0.752506,0.205623,0.764903
min,-5.489590,-6.686231,-1.577246,-1.207290,-0.729410,-11.146330,-0.426893,-0.218267,-0.303612
25%,-0.639742,-0.617604,-0.772966,-0.537198,-0.687225,-0.884367,-0.423204,-0.054200,-0.287010
50%,-0.025013,-0.045533,-0.387381,-0.153068,-0.384559,-0.084398,-0.337888,0.059176,-0.245017
75%,0.708696,0.549955,0.156282,0.493550,0.271357,0.000000,0.018240,0.226527,-0.108894
max,5.572708,7.618869,3.065788,12.640583,12.174918,0.000000,23.336675,1.745879,45.855204


<a id='time_history'></a>
### 2.3.5 Adding Location and Lag Times

We add in the sampling location from the THEMIS data and the time history of all of the driving features in `features_normed`.

In [23]:
# This is where the DataFrame will be saved.
feature_array_fname = './Data/Training/full_feature_array.pkl'
# The method appends the year in yyyy format to the end of the filename.
# The above becomes './Data/Training/full_feature_array_2021.pkl'
# This was useful when working with data from several years.

# This method may take several minutes depending on the length.
omni_data.filter_features(
    feature_normed_fname,
    position_fname,
    feature_array_fname,
    )

Estimated time remaining: 1 min, 27 sec
Estimated time remaining: 1 min, 12 sec
Estimated time remaining: 1 min, 9 sec
Estimated time remaining: 1 min, 5 sec
Estimated time remaining: 1 min, 1 sec
Estimated time remaining: 0 min, 58 sec
Estimated time remaining: 0 min, 54 sec
Estimated time remaining: 0 min, 50 sec
Estimated time remaining: 0 min, 46 sec
Estimated time remaining: 0 min, 42 sec
Estimated time remaining: 0 min, 38 sec
Estimated time remaining: 0 min, 34 sec
Estimated time remaining: 0 min, 30 sec
Estimated time remaining: 0 min, 27 sec
Estimated time remaining: 0 min, 23 sec
Estimated time remaining: 0 min, 19 sec
Estimated time remaining: 0 min, 15 sec
Estimated time remaining: 0 min, 11 sec
Estimated time remaining: 0 min, 7 sec
Estimated time remaining: 0 min, 3 sec
Master Inputs DataFrame is using 0.26579008 GiB of memory
Saved data for year 2021.


<a id='model_ready_data'></a>
### 2.4 Model Ready Data

We finally create arrays that are ready to be used for training the neural network.
In the full model, the training/validation/testing split was done by years. In this example, we simply split by fractions, randomly.

In [24]:
feature_array_fname_as_saved = './Data/Training/full_feature_array_2021.pkl'
target_df_fname = './Data/Training/target_df.pkl'
modelready_dir = './Data/Training/ModelReady/'

splits = {
    'train' : 0.8,
    'val' : 0.1,
    'test' : 0.1
    }

tdata = td.TrainingData()

tdata.create_training_arrays_by_fraction(
    feature_array_fname_as_saved,
    target_df_fname,
    modelready_dir,
    splits
    )

<a id='neural_network'></a>
## 3. Artificial Neural Network

<a id='optimization'></a>
### 3.1 Hyperparameter Optimization

This will take several minutes.

In [29]:
import optimize_parameters as optpara
optpara.optimize()

>>> Imports:
#coding=utf-8

try:
    import pickle
except:
    pass

try:
    from pandas import read_hdf
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Dropout
except:
    pass

try:
    from tensorflow.keras import optimizers, losses, metrics
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas.distributions import choice, uniform, quniform, loguniform
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from nnmodel import optimize_model
except:
    pass

try:
    from metrics import MSA_with_log10
except:
    pass

try:
    from pathlib import Path
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'units': hp.quniform('units', 32, 1024, 32),
        'Dropout': hp.uniform('Dropout', 0, 1),
        'units_1': hp.quniform('units_1', 32, 1024, 32),
        'Dropout_1': hp.un

2022-05-06 16:25:39.428986: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 16:25:39.444172: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa935ffd800 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-06 16:25:39.444182: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


100%|███████████████████| 10/10 [03:35<00:00, 21.53s/trial, best loss: 102.25649958069658]
Evaluation of best performing model:
222/222 [==============================] - 0s 882us/step - loss: 0.1247
0.12470220029354095
Best performing model chosen hyper-parameters:
{'Dropout': 0.30567409873485496, 'Dropout_1': 0.04864354429440787, 'Dropout_2': 0.6357875518823483, 'batch_size': 10000, 'beta_1': 0.8662237464555252, 'beta_1_1': 0.9709155109822891, 'delta': 1.5369221073953567, 'learning_rate': 0.01, 'units': 992.0, 'units_1': 320.0, 'units_2': 352.0}


<a id='nn_training'></a>
### 3.2 Model Training

Technically, the optimal model is the one found during hyperparameter optimization. So, we can retrain a model using those parameters, or we can just use the last model that it ran.
The last model was saved to disk as `best_model_example.h5` in the save folder (default is `./Data/Optimizations/`).

<a id='predictions'></a>
## 4. Make Predictions

In [30]:
# First we load the trained model.
from tensorflow.keras.models import load_model

model_fname = './Data/Optimizations/best_model_example.h5'

model = load_model(model_fname)

In [31]:
# Then we make predictions using the test data.
test_features = pd.read_hdf('./Data/Training/ModelReady/test_features.h5')

predictions = model.predict(test_features.values)

In [32]:
# The predictions are a numpy array. We like pandas DataFrames.
test_targets = pd.read_hdf('./Data/Training/ModelReady/test_targets.h5')

predictions = pd.DataFrame(
    data=predictions,
    index=test_targets.index,
    columns=test_targets.columns,
    )

In [33]:
predictions.head()

,82.5,108.7,142.5,187.4,247.0,324.7,427.6,562.1,740.4,975.4,...,26388.6,27000.0,28000.0,29000.0,30000.0,31000.0,41000.0,52000.0,65500.0,93000.0
2021-06-06 12:58:00,6.988613,6.997972,7.003815,7.015029,7.013207,7.016743,7.005485,7.004931,6.989583,6.974095,...,4.040700,4.003200,3.950647,3.925737,3.921461,3.969457,3.123244,2.698105,2.301425,1.885583
2021-06-13 05:40:00,6.688765,6.750711,6.798782,6.834392,6.849105,6.851921,6.867518,6.867037,6.867595,6.861219,...,4.812826,4.729975,4.610234,4.489483,4.408675,4.333295,3.654217,3.254342,2.890587,2.439238
2021-05-12 09:37:00,7.525219,7.545461,7.584201,7.567977,7.602909,7.621081,7.600132,7.606915,7.569940,7.560345,...,4.505274,4.461185,4.374432,4.330528,4.291373,4.336465,3.472305,2.994184,2.586319,2.152872
2021-06-15 16:05:00,7.196827,7.232785,7.277373,7.295441,7.332248,7.340070,7.327604,7.335577,7.314184,7.327237,...,4.874776,4.803299,4.673414,4.573879,4.485277,4.449751,3.702365,3.262784,2.874431,2.420771
2021-05-02 03:58:00,6.727911,6.819920,6.881334,6.910787,6.943572,6.962813,6.969631,6.982332,6.981168,6.974397,...,5.133865,5.041938,4.891342,4.746273,4.626608,4.520013,3.877736,3.471198,3.114594,2.666708


<a id='assessment'></a>
## 5. Model Assessment

We assess the model's performance using several metrics.

<a id='metrics'></a>
### 5.1 Metrics

In [34]:
import metrics as mc

In [35]:
# The method takes a dictionary of energy channels; can customize the labels if desired.
# We will just use the column names here.
energy_channel_dict = dict(zip(test_targets.columns, test_targets.columns))

fit_metrics = mc.combine_fit_metrics(
    energy_channel_dict,
    test_targets,
    predictions,
    )
    

In [36]:
fit_metrics

,R_log,MAPE_log,sMAPE_log,PE_log,MSE_log,MAE_log,ME_log,MSA,SSPB,IP,obs_var_log,pred_var_log,obs_var,pred_var
82.5,0.548220,6.840650,6.872012,0.227690,0.334250,0.473661,-0.113739,164.708212,-35.807404,35.669059,0.432793,0.050309,2.591323e+15,2.357473e+13
108.7,0.573050,5.943521,5.976917,0.240601,0.265560,0.415002,-0.110241,128.764384,-31.581873,41.972703,0.349697,0.041085,2.001658e+15,2.311652e+13
142.5,0.590445,5.310819,5.345197,0.252083,0.222503,0.372887,-0.112841,105.388166,-31.890778,48.388912,0.297497,0.038261,1.542978e+15,2.507958e+13
187.4,0.610887,4.877178,4.901535,0.261252,0.197058,0.342683,-0.111011,88.801424,-31.559947,54.031237,0.266746,0.033523,1.167160e+15,2.253918e+13
247.0,0.620437,4.561600,4.565436,0.278477,0.181516,0.318821,-0.108071,76.995283,-29.102977,58.266498,0.251574,0.035454,8.759641e+14,2.645823e+13
324.7,0.627718,4.338916,4.325972,0.295710,0.171910,0.301291,-0.105681,67.706848,-29.346379,62.121852,0.244089,0.038752,6.607205e+14,2.958519e+13
427.6,0.619942,4.349676,4.307654,0.277126,0.177002,0.298631,-0.103188,66.943895,-31.268398,63.163079,0.244858,0.033073,5.049601e+14,2.484362e+13
562.1,0.615433,4.259998,4.206421,0.282963,0.172454,0.290525,-0.099673,63.697585,-33.047431,65.301815,0.240509,0.035097,3.878163e+14,2.643426e+13
740.4,0.596285,4.355765,4.269948,0.258162,0.180686,0.293001,-0.100776,65.027762,-38.058768,64.457577,0.243565,0.031615,2.890133e+14,2.244627e+13
975.4,0.575840,4.453774,4.339785,0.245369,0.186007,0.295697,-0.100524,66.543224,-42.558238,63.908822,0.246488,0.032510,2.065116e+14,2.275281e+13
